In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

def process_image(image_path, pixel_to_um=2):
    try:
        print(f"Reading image from {image_path}")
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(f"Failed to read image from {image_path}")
            return [], [], None
        
        print("Applying Gaussian Blur")
        blurred = cv2.GaussianBlur(image, (15, 15), 0)
        
        print("Applying high pass filter")
        high_pass_kernel = np.array([[-1, -1, -1],
                                     [-1,  8, -1],
                                     [-1, -1, -1]])
        high_pass_filtered = cv2.filter2D(blurred, -2, high_pass_kernel)
        high_pass_filtered = cv2.normalize(high_pass_filtered, None, 0, 255, cv2.NORM_MINMAX)

        print("Applying thresholding")
        _, binary = cv2.threshold(high_pass_filtered, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        dist_transform = cv2.distanceTransform(binary, cv2.DIST_L2, 5)
        _, markers = cv2.threshold(dist_transform, 0.3 * dist_transform.max(), 255, 0)
        markers = np.uint8(markers)

        print("Identifying sure background and sure foreground areas")
        sure_bg = cv2.dilate(binary, np.ones((3, 3), np.uint8), iterations=5)
        sure_fg = cv2.erode(binary, np.ones((3, 3), np.uint8), iterations=5)
        unknown = cv2.subtract(sure_bg, sure_fg)

        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0

        print("Applying watershed algorithm")
        color_image = cv2.cvtColor(high_pass_filtered, cv2.COLOR_GRAY2BGR)
        cv2.watershed(color_image, markers)
        color_image[markers == -1] = [0, 255, 0]

        print("Processing regions and contours")
        gray_color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray_color_image, (5, 5), 0)
        binary = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

        binary[:40, :] = 0
        binary[-40:, :] = 0
        binary[:, :20] = 0
        binary[:, -20:] = 0

        num_labels, labels = cv2.connectedComponents(binary)
        area_threshold = 70

        filtered_labels = np.zeros_like(labels)
        for i in range(1, num_labels):
            cell_mask = labels == i
            if np.sum(cell_mask) >= area_threshold:
                filtered_labels[cell_mask] = i

        print("Splitting cells")
        split_labels = split_cells(filtered_labels, area_threshold)

        for i in range(1, split_labels.max() + 1):
            cell_mask = split_labels == i
            if np.any(cell_mask):
                filled_cell = ndi.binary_fill_holes(cell_mask).astype(int)
                split_labels[filled_cell > 0] = i

        colored_cells = np.zeros((split_labels.shape[0], split_labels.shape[1], 3), dtype=np.uint8)
        for i in range(1, split_labels.max() + 1):
            cell_mask = split_labels == i
            if np.any(cell_mask):
                contours, _ = cv2.findContours(cell_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                color = np.random.randint(0, 255, size=3).tolist()
                cv2.drawContours(colored_cells, contours, -1, color, -1)

        gray = cv2.cvtColor(colored_cells, cv2.COLOR_BGR2GRAY)
        _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        solid_contours = [cnt for cnt in contours if is_solid_region(cnt)]

        print("Calculating region properties")
        regions_properties = []
        for cnt in solid_contours:
            area = cv2.contourArea(cnt) * (pixel_to_um ** 2)
            perimeter = cv2.arcLength(cnt, True) * pixel_to_um
            equivalent_diameter = np.sqrt(4 * area / np.pi)
            area_weighted_equivalent_diameter = area * equivalent_diameter
            x, y, w, h = cv2.boundingRect(cnt)
            min_diameter = min(w, h) * pixel_to_um
            max_diameter = max(w, h) * pixel_to_um
            mean_diameter = np.mean([w, h]) * pixel_to_um
            aspect_ratio = w / h
            elongation = 1 - (w / h)
            if len(cnt) >= 5:
                ellipse = cv2.fitEllipse(cnt)
                (major_axis, minor_axis), angle = ellipse[1], ellipse[2]
                orientation = angle
            else:
                orientation = np.nan
            roundness = (4 * area) / (np.pi * (mean_diameter ** 2))
            centroid = (x + w // 2, y + h // 2)
            regions_properties.append({
                'Projected area (µm²)': area,
                'Perimeter (µm)': perimeter,
                'Equivalent diameter (µm)': equivalent_diameter,
                'Area weighted equivalent diameter (µm)': area_weighted_equivalent_diameter,
                'Mean diameter (µm)': mean_diameter,
                'Minimum diameter (µm)': min_diameter,
                'Maximum diameter (µm)': max_diameter,
                'Length (µm)': h * pixel_to_um,
                'Width (µm)': w * pixel_to_um,
                'Aspect ratio': aspect_ratio,
                'Roundness': roundness,
                'Elongation': elongation,
                'Orientation (°)': orientation,
                'Centroid': centroid
            })

        print(f"Processed {len(regions_properties)} regions")
        return regions_properties, solid_contours, color_image
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return [], [], None

def split_cells(labels, area_threshold):
    new_labels = np.zeros_like(labels)
    current_label = 1
    for i in range(1, labels.max() + 1):
        cell_mask = labels == i
        if np.sum(cell_mask) >= area_threshold:
            dist_transform = cv2.distanceTransform(cell_mask.astype(np.uint8), cv2.DIST_L2, 5)
            _, split_markers = cv2.threshold(dist_transform, 0.2 * dist_transform.max(), 255, 0)
            split_markers = np.uint8(split_markers)
            num_splits, split_labels = cv2.connectedComponents(split_markers)
            for j in range(1, num_splits):
                split_mask = split_labels == j
                if np.sum(split_mask) >= area_threshold:
                    new_labels[split_mask] = current_label
                    current_label += 1
        else:
            new_labels[cell_mask] = current_label
            current_label += 1
    return new_labels

def is_solid_region(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    return area > 100 and perimeter / area < 0.1

def plot_and_save_image(image, contours, centroids, output_path):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    for contour in contours:
        plt.plot(contour[:, 0, 0], contour[:, 0, 1], 'b-', linewidth=1)
    
    for centroid in centroids:
        plt.plot(centroid[0], centroid[1], 'r.', markersize=10)
    
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()

def test_image_processing(image_path):
    try:
        base_dir = os.path.dirname(image_path)
        export_dir = os.path.join(base_dir, 'Exported Results')
        os.makedirs(export_dir, exist_ok=True)
        
        pixel_to_um = 2
        properties, contours, processed_image = process_image(image_path, pixel_to_um)
        
        if processed_image is None:
            print(f"Error processing {image_path}: processed_image is None")
            return
        
        properties_df = pd.DataFrame(properties)
    
        output_image_path = os.path.join(export_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_processed.png")
        centroids = [prop['Centroid'] for prop in properties]
        plot_and_save_image(processed_image, contours, centroids, output_image_path)
    
        print(f"Processed image: {image_path}")
        print(f"Exported processed image to: {output_image_path}")
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")

# Example usage:
image_path = r'C:\Users\chilukalo\OneDrive - Carlisle Global\Desktop\SEM\Mike Test\Capture.PNG'
test_image_processing(image_path)


Reading image from C:\Users\chilukalo\OneDrive - Carlisle Global\Desktop\SEM\Mike Test\Capture.PNG
Applying Gaussian Blur
Applying high pass filter
Applying thresholding
Identifying sure background and sure foreground areas
Applying watershed algorithm
Processing regions and contours
Splitting cells
Calculating region properties
Processed 8 regions
Processed image: C:\Users\chilukalo\OneDrive - Carlisle Global\Desktop\SEM\Mike Test\Capture.PNG
Exported processed image to: C:\Users\chilukalo\OneDrive - Carlisle Global\Desktop\SEM\Mike Test\Exported Results\Capture_processed.png
